In [1]:
import os
import gym                                                      # Build envs and load pre-existing envs        
from stable_baselines3 import PPO                               # RL Algorithm
from stable_baselines3.common.vec_env import DummyVecEnv        # Vectorize environments
from stable_baselines3.common.evaluation import evaluate_policy

import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

## Loading an environment

In [2]:
environment_name = 'CartPole-v1'
env = gym.make(environment_name, render_mode="human")

In [3]:
episodes = 5

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        next_obs_state, reward, done, trunc, info = env.step(action)
        score += reward

    print(f"Episode: {episode} | Score: {score}")
env.close()

Episode: 1 | Score: 13.0
Episode: 2 | Score: 10.0
Episode: 3 | Score: 34.0
Episode: 4 | Score: 29.0
Episode: 5 | Score: 18.0


## Understanding the Environment

In [4]:
env.action_space

Discrete(2)

In [5]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

## Train an RL Model

In [6]:
log_path = os.path.join('training', 'logs')

In [7]:
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [8]:
model.learn(total_timesteps=20000)

Logging to training\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 46   |
|    iterations      | 1    |
|    time_elapsed    | 44   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 46          |
|    iterations           | 2           |
|    time_elapsed         | 88          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008233212 |
|    clip_fraction        | 0.088       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | 0.00149     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.37        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0126     |
|    value_loss           | 46.1        |
-----------------------------------------
---

## Save and load the model

In [9]:
save_path = os.path.join('training', 'saved_models', 'PPO_model_cartpole')

In [10]:
model.save(save_path)

In [11]:
loaded_model = PPO.load(save_path, env=env)
loaded_model

## Evaluation

In [12]:
print(evaluate_policy(model, env, n_eval_episodes=10, render=True))     # Mean / Std (time upright)

(486.5, 31.784430150625635)


## Testing

In [13]:
episodes = 5

for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward

    print(f"Episode: {episode} | Score: {score}")
env.close()

Episode: 1 | Score: [58.]
Episode: 2 | Score: [188.]
Episode: 3 | Score: [400.]
Episode: 4 | Score: [210.]
Episode: 5 | Score: [362.]
